# Description

This notebook provides the code to re-produce figures and tables for *The Lancet Countdown on Health and Climate Change* **Indicator 5.3.3: Political party engagement with health and climate change**

**Authors:** Zach Dickson & Cornelius Erfort 

In [8]:
!pip install -q plotly pandas numpy nbformat kaleido scikit-learn matplotlib seaborn

## Primary Figure: Indicator 5.3.3: Political party engagement with health and climate change

In [1]:
# import libraries
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


# read in data 
df = pd.read_csv('../data/indicator_5_3_3.csv')
# drop values after 2025 
df = df[df['year'] <= 2025]

# dictionary to map party names to respective party colors
party_colors = {'Christian Democracy': '#1f77b4',
                'Conservative': '#9D755D',
                'Liberal': '#EECA3B',
                'Green': '#54A24B',
                'Radical Right-Wing': '#222A2A',
                'Social Democracy': '#E45756',}


# Define the mapping of data column -> subplot position
col_to_position = {
    'environment_climate_issue1_mean': (1, 1),  # Climate Change
    'healthcare_issue1_mean': (1, 2),           # Public Health
    'climate_health_mean': (2, 1),              # Climate and Health Nexus (mean)
    'climate_health_sum': (2, 2)                # Climate and Health Nexus (sum)
}


def create_nexus_subplot(df, 
                         title = 'Party Press Releases on Climate Change and Health by Party Family',
                         save_path=None,
                         col_to_position = col_to_position,
                         party_colors = party_colors):

    # Titles should match the order of the subplot grid
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Climate Change',
            'Public Health',
            'Climate & Health Nexus (mean)', 
            'Climate & Health Nexus (sum)'
        ),
        horizontal_spacing=0.05,  # reduce horizontal gap between columns
        vertical_spacing=0.15      # reduce vertical gap between rows
    )

    # Loop through the variables and plot each one in its correct subplot
    for i, col in enumerate(col_to_position.keys()):
        row, col_num = col_to_position[col]
        for party_family in party_colors.keys():
            subset = df[df['party_family'] == party_family].copy()

            # smooth the line
            subset[col] = subset[col].rolling(window=2, min_periods=1).mean()
            subset[col] = subset[col].rolling(window=2, min_periods=1).mean()

            fig.add_trace(go.Scatter(
                x=subset['year'],
                y=subset[col],
                mode='lines+markers',
                name=party_family,
                line=dict(color=party_colors[party_family], width=5),
                showlegend=True if i == 0 else False,
            ), row=row, col=col_num)

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Year',
        yaxis_title='Press Releases',
        title_x=0.05,
        legend_title='Party Family',
        hovermode='x unified',
        width=1000,
        height=700,
        template='presentation',
        font=dict(size=14),
        margin=dict(l=70, r=50, t=100, b=60)
    )

    # Format all y-axes as percentages
    fig.update_yaxes(tickformat=".0%", row=1, col=1)
    fig.update_yaxes(tickformat=".0%", row=1, col=2)
    fig.update_yaxes(tickformat=".0%", row=2, col=1)

    # Custom x-axis titles per subplot 
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_xaxes(title_text="Year", row=2, col=2)
    fig.update_xaxes(title_text="Year", row=1, col=2)

    # add y-axis titles
    fig.update_yaxes(title_text="Press Releases", row=1, col=1)
    #fig.update_yaxes(title_text="Press Releases", row=1, col=2)
    fig.update_yaxes(title_text="Press Releases", row=2, col=1)
    #fig.update_yaxes(title_text="Press Releases", row=2, col=2)


    # save figure if save_path is provided
    if save_path:
        fig.write_image(save_path, scale=2)
    # show figure
    fig.show()


create_nexus_subplot(df, 
                     #title='',
                     save_path='../appendix/indicator_5_3_3.png',
                     col_to_position=col_to_position,
                     party_colors=party_colors)


# Create table for appendix

In [2]:
# import libraries for classification report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# read in validation data 
test = pd.read_parquet('../data/llm_validation_data.parquet')

# print classification report
print(classification_report(test['labels'].values, test.predicted_labels_integer.values))

# create confusion matrix
cm = confusion_matrix(test['labels'].values, test.predicted_labels_integer.values)

fig = px.imshow(cm,
                text_auto=True, 
                aspect="auto",
                color_continuous_scale='Blues',
                labels=dict(x="Predicted Label", y="True Label", color="Count"),
                title="Confusion Matrix",
)
fig.update_layout(
    height=600,
    width=600,
)
fig.show()

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       308
           1       0.93      0.91      0.92        69
           2       0.97      0.97      0.97       195
           3       0.96      0.97      0.97        79
           4       0.90      0.95      0.93       132
           5       0.97      0.95      0.96       157
           6       0.97      0.94      0.95       168
           7       0.84      0.97      0.90        33
           8       0.95      0.94      0.95        88
           9       0.94      0.97      0.96        69
          10       0.95      0.95      0.95        79
          11       0.90      0.87      0.88        53
          12       0.94      0.96      0.95        48
          13       0.93      0.81      0.87        16
          14       0.84      0.97      0.90        80
          15       0.71      1.00      0.83         5
          16       0.92      0.69      0.79        16
          17       0.92    